# **Analysis of Renewable Energy Sector in Europe**

This project explores trends in renewable energy sector in Europe based on annual data covering 1985 till 2024. It focuses on primary energy production statistics, imports, exports, gross available energy and final energy consumption. All data is publicly available via Our World in Data, Eurostat and Energy Institute.


### Table of Contents
* Goals
* Highlights
* Energy Consumption by Source: dataset overview
* Key turning points of six decades in a nutshell and spotted trends
* Energy Production by Source: dataset overview
* Future


### Goals
The goal of this project was to hone Python, Jupyter Notebook and CSV file handling skills by tackling a problem and answering questions along the way. For this, a project with a focus on renewable energies was chosen, as it is a field I am fascinated and interested in. To narrow it down, the European continent was chosen as a primary focus.

I would like to answer the following questions in this project:
* What are the main energy sources?
* ~~What trends energy sector has been facing in a past six decades?~~
* ~~Where renewable energy stands~~ in terms of production and consumption today?
* How do countries compare when we look at total electricity consumption.
* How much of Europe's energy comes from low-carbon sources?
* Which source of renewable energy are growing the fastest in terms of expansion, innovation and investments?


### Highlights
* The European energy sector has transformed since 1965, and we can see this change in primary energy consumption and generation. While in 1965 the two primary sources of energy were oil and coal, if we look into 2024 data, renewable energy's consumption is increasingly displacing them, with coal consumption lowering each year.
* Wind energy
*



##### _Libraries for data loading and inspection_

In [56]:
import pandas as pd
import plotly.express as px
import unittest

from matplotlib import pyplot as plt
%matplotlib inline


### Energy Consumption by Source dataset overview

Before focusing solely on renewables, first I need to explore ~~what energy sources Europe consume~~ in general and where renewables stand in the energy industry.

In [60]:
share_energy = pd.read_csv('../data/energy_consumption_by_source_europe.csv')
share_energy.head()

,Entity,Code,Year,other_renewables_consumption_equivalent_twh,biofuels_consumption_twh,solar_consumption_equivalent_twh,wind_consumption_equivalent_twh,hydro_consumption_equivalent_twh,nuclear_consumption_equivalent_twh,gas_consumption_twh,coal_consumption_twh,oil_consumption_twh
0,Austria,AUT,1965,0.0,0.0,0.0,0.0,44.675,0.0,17.528648,59.114902,65.29748
1,Austria,AUT,1966,0.0,0.0,0.0,0.0,48.141663,0.0,18.821033,56.467594,71.788864
2,Austria,AUT,1967,0.0,0.0,0.0,0.0,49.16111,0.0,18.031685,53.4509,76.54811
3,Austria,AUT,1968,0.0,0.0,0.0,0.0,50.513885,0.0,19.22897,52.77368,88.466064
4,Austria,AUT,1969,0.0,0.0,0.0,0.0,46.43889,0.0,23.329157,51.9364,97.226


In this chart, we can see the breakdown of European primary energy consumption based on source, measured in terms of primary energy using the substitution method. This dataset is sourced from the website "Our World in Data", "Energy consumption by source, Europe".<sup id="fnref1"><a href="#fn1">1</a></sup> There was no available data for the following countries: Andorra, Liechtenstein, Monaco, San Marino and Vatican City. Numbers are given in terms of per capita consumption. **< make sure you mention how this transforms into total consumption, this is a bit confusing** Here is the quick summary of the dataset columns:
* **Entity**: the name of the country.
* **Code**: country code.
* **Year**: the year data represents.
* **other_renewables_consumption_equivalent_twh**: energy consumption from geothermal, biomass, waste, wave, and tidal source in Terawatt-hours.
* **biofuels_consumption_twh**: Biofuel consumption in Terawatt-hours.
* **solar_consumption_equivalent_twh**: Solar consumption in Terawatt-hours.
* **wind_consumption_equivalent_twh**: Wind consumption in Terawatt-hours.
* **hydro_consumption_equivalent_twh**: Hydro consumption in Terawatt-hours.
* **nuclear_consumption_equivalent_twh**: Nuclear consumption in Terawatt-hours.
* **gas_consumption_twh**: Gas consumption in Terawatt-hours.
* **coal_consumption_twh**: Coal consumption in Terawatt-hours
* **oil_consumption_twh**: Oil consumption in Terawatt-hours.

<b id="fn1">1.</b> _Data source: Energy Institute - Statistical Review of World Energy (2025) - with major processing by Our World in Data_ <a href="https://ourworldindata.org/electricity-mix"> link </a>

In [62]:
# === Clean column names ===
share_energy.columns = share_energy.columns.str.strip()

# === Define energy columns in desired stack order (bottom to top) ===
energy_cols = [
    "other_renewables_consumption_equivalent_twh",
    "biofuels_consumption_twh",
    "solar_consumption_equivalent_twh",
    "wind_consumption_equivalent_twh",
    "hydro_consumption_equivalent_twh",
    "nuclear_consumption_equivalent_twh",
    "coal_consumption_twh",
    "gas_consumption_twh",
    "oil_consumption_twh"
]

# === Ensure numeric values (keep NaN for missing data) ===
share_energy[energy_cols] = share_energy[energy_cols].apply(pd.to_numeric, errors='coerce')

# === Count available countries per year ===
country_counts = share_energy.groupby("Year")["Entity"].count().to_dict()

# === Aggregate European countries into ONE row per year, sum available values only ===
share_energy_agg = share_energy.groupby(["Year"], as_index=False)[energy_cols].sum()

# === Compute total energy per row ===
share_energy_agg['Total_energy'] = share_energy_agg[energy_cols].sum(axis=1).round(2)

# === Prepare long-format data for Plotly ===
df_melted = share_energy_agg.melt(
    id_vars=['Year', 'Total_energy'],
    value_vars=energy_cols,
    var_name='Source',
    value_name='Energy_TWh'
)

# === Clean source names for display ===
source_name_map = {
    "other_renewables_consumption_equivalent_twh": "Other renewables",
    "biofuels_consumption_twh": "Biofuels",
    "solar_consumption_equivalent_twh": "Solar",
    "wind_consumption_equivalent_twh": "Wind",
    "hydro_consumption_equivalent_twh": "Hydro",
    "nuclear_consumption_equivalent_twh": "Nuclear",
    "gas_consumption_twh": "Gas",
    "oil_consumption_twh": "Oil",
    "coal_consumption_twh": "Coal"
}
df_melted['Source'] = df_melted['Source'].replace(source_name_map)

# === Reverse stack order for plot and legend ===
stack_order = [source_name_map[col] for col in energy_cols][::-1]  # bottom-to-top
df_melted['Source'] = pd.Categorical(df_melted['Source'], categories=stack_order, ordered=True)

# === Generate hover text per year including number of countries ===
hover_text_dict = {}
for year in share_energy_agg['Year'].unique():
    row = share_energy_agg[share_energy_agg['Year'] == year]
    hover_text = f"<b>Year: {year}</b><br>"
    hover_text += f"Countries with data: {country_counts.get(year, 0)}<br>"
    for col in energy_cols[::-1]:
        display_name = source_name_map[col]
        hover_text += f"{display_name}: {row[col].values[0]:,.2f} TWh<br>"
    hover_text += f"<b>Total: {row['Total_energy'].values[0]:,.2f} TWh</b>"
    hover_text_dict[year] = hover_text

df_melted['hover_text'] = df_melted['Year'].map(hover_text_dict)

# === Create Plotly area chart ===
fig = px.area(
    df_melted,
    x='Year',
    y='Energy_TWh',
    color='Source',
    category_orders={'Source': stack_order},
    hover_data={'Energy_TWh': False, 'Total_energy': False},
    title='Europe: Energy Consumption by Source (1965–2024)'
)

fig.update_traces(
    text=df_melted['hover_text'],
    hovertemplate="%{text}<extra></extra>"
)

# === Update layout ===
fig.update_layout(
    yaxis_title='Energy Consumption (TWh)',
    xaxis_title='Year',
    yaxis=dict(range=[0, 37000]),
    xaxis=dict(range=[1965, 2024]),
    template='plotly_white',
    legend_traceorder="reversed"
)

fig.show()

# class TestEnergyData(unittest.TestCase):
#
#     def setUp(self):
#         # Example: your DataFrame
#         # Replace this with your actual DataFrame variable
#         self.df = share_energy.copy()
#
#         # Columns to test
#         self.energy_cols = [
#             "Other renewables (including geothermal and biomass) - TWh",
#             "Biofuels consumption - TWh",
#             "Solar consumption - TWh",
#             "Wind consumption - TWh",
#             "Hydro consumption - TWh",
#             "Nuclear consumption - TWh",
#             "Gas consumption - TWh",
#             "Oil consumption - TWh"
#         ]
#
#     def test_columns_exist(self):
#         # Ensure all expected columns exist
#         for col in self.energy_cols + ["Year"]:
#             self.assertIn(col, self.df.columns)
#
#     def test_no_nan_values(self):
#         # Check that energy columns have no NaN
#         self.assertFalse(self.df[self.energy_cols].isna().any().any())
#
#     def test_numeric_values(self):
#         # Ensure all energy columns are numeric
#         for col in self.energy_cols:
#             self.assertTrue(np.issubdtype(self.df[col].dtype, np.number))
#
#     def test_total_energy(self):
#         # Compute total per row
#         self.df['Total_energy'] = self.df[self.energy_cols].sum(axis=1).round(2)
#         # Check if total energy is within reasonable range
#         # Example: global energy generation should not exceed 100,000 TWh/year
#         self.assertTrue((self.df['Total_energy'] >= 0).all())
#         self.assertTrue((self.df['Total_energy'] <= 100000).all())
#
#     def test_shape(self):
#         # Check if dataframe has expected columns and rows > 0
#         self.assertGreater(self.df.shape[0], 0)
#         self.assertGreaterEqual(self.df.shape[1], len(self.energy_cols) + 1)  # +1 for Year
#
# suite = unittest.TestLoader().loadTestsFromTestCase(TestEnergyData)
# unittest.TextTestRunner(verbosity=2).run(suite)

Because used data set contains no negative values and the goal was to represent a total energy consumption and visualise how much of that energy has been generated by each listed source, I have used stacked area graph. To make sure that no values are off and data is handled properly, I will use an unittest alongside for this and other data visuals to practice test implementation in my code.

#### Key turning points of six decades in a nutshell and spotted trends:
* **Between 1965 and 1975**, market was dominated by fossil fuels, and oil based power consumption points towards steady growth from 36.10 % in 1965 to 44.41 % by 1975 showing a 23.02 % increase.
* **Between 1975 and 1990**, due to 1973 oil crisis caused by OPEC energy sector starts to diversify. Due to nuclear technology rapid advancement and more power plants being build in Europe, nuclear power consumption starts to grow.
* **Between 1990 and 2005**, the renewable energy sector grew significantly, moving from a nascent stage to a more established and growing industry, largely driven by concerns about fossil fuels and environmental impacts. This can be seen by steadily growing wind energy, biofuel consumption and other renewables such as geothermal energy.
* **Between 2005 and 2016**, global investments and declining costs for solar panel and wind turbine production, accompanied by subsidies within EU, made renewable energy consumption reach record high in 2006, and by 2016 renewable energy was increasingly displacing fossil fuels accompanied by nuclear power in the energy mix.
* **Between 2015 and 2024**, motivated by the European Green Deal and rise of renewables, consumption of coal based energy is on steady decline. Meanwhile from 2015 to 2024, the renewable energy consumption increased from 12.42 % to 17.85 % or 43.7 %.
*  There is a clear steep increase in electricity consumption from 1984 till 1985 based on Energy Institute and Our World In Data datasets.
* There is an overall trend in electricity consumption decline starting from 1990, which is caused primarily by investments in modernisation of power grids and alternative energy sources from fossil fuels.
* On a global level Europe accounted for one-quarter of world primary energy production in 2002 (WE 10.4%, CESE 2.2% and R&CIS 12.4%)
* The exploitation of renewables varies, however, from country to country - mainly because of diverse geographical and climatic conditions, but also because of policy differences. Variations are also noteworthy with regard to nuclear energy. Whereas nuclear energy consumption is nil in some countries, the share of nuclear energy in total consumption is quite substantial in others, such as in France where the nuclear sector represents as much 37% of total primary energy consumption and generates 77% of all electricity.

#### Sources:
* _Office of The Historian - Oil Embargo, 1973-1974: https://history.state.gov/milestones/1969-1976/oil-embargo_
* _Our World in Data — Energy Mix article: https://ourworldindata.org/electricity-mix_
* _Wikipedia - History of nuclear power: https://en.wikipedia.org/wiki/History_of_nuclear_power_
* _European Environment Agency - Renewables accounted for vast majority of new EU power capacity in 2016: https://www.eea.europa.eu/highlights/renewables-accounted-for-vast-majority_
* _Eurostat — Energy Statistics – An Overview: https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Energy_statistics_-_an_overview_
* _Eurostat - Renewable energy supply grew by 3.4% in 2024: https://ec.europa.eu/eurostat/web/products-eurostat-news/w/ddn-20250702-1_


### Energy Production by Source dataset overview

After an analysis on energy consumption by source in Europe, I looked into electricity production from renewable sources dataset to have a better overview on each European countries renewable energy sector. However, there was no available data for the following countries: Andorra, Liechtenstein, Monaco, San Marino and Vatican City.

Here is the quick summary of the dataset columns:
* **Entity**: the name of the country.
* **Code**: a unique code Energy Institute (EI) uses for each country. Since this information is not publicly shared, all the values are listed as unknown.
* **Year**: the year data represents.
* **renewable_share_of_electricity__pct**: a share of electricity production from renewable sources in percentage.

### Future
* An action in modernisation of power sector and investments in renewables have a visible effect seen in increased green energy consumption and production. However, current global climate and policies are driving Europe to be still heavily fossil fuel dominated in 2030. This issue can be tackled by supporting research on renewable energy, resource development, strategic reserves, investment in green power generation, stronger fossil fuel regulations and reducing reliance on a single supplier or power source. For instance, European energy sector is still vulnerable against fluctuating oil or gas prices, and geopolitical instability as shown in 2022 with Russian annexation of Ukraine.
* While most European countries are experiencing declining energy consumption due to climate change or modernisation, this decline is slow.
* The biggest energy user are households.
